#### 크롤링

In [126]:
import pandas as pd
import requests as req
from requests_toolbelt.utils import dump
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains 
from bs4 import BeautifulSoup as bs
import time

In [275]:
import pandas as pd
from datetime import datetime

In [233]:
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

# 사이트에 접속
driver = webdriver.Chrome()
driver.get("https://m.humoruniv.com/main.html")

time.sleep(3)

# 검색 후 다시 경고창 확인하고 닫기
try:
    alert = Alert(driver)
    alert.accept()  # 경고창 확인 버튼을 클릭
except:
    pass  # 경고창이 없는 경우 패스

button = driver.find_element(By.CLASS_NAME, 'hu_icon')

# 버튼 클릭
button.click()

# 클릭 후 잠시 대기
time.sleep(3)

# 검색창에 텍스트 입력
pyperclip.copy("공강")
search = driver.find_element(By.NAME, 'search_text')
search.clear()
search.send_keys(Keys.CONTROL, 'v')  # Ctrl + V로 붙여넣기

# 잠시 대기
time.sleep(3)

# 검색 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, "input[type='submit']")
search_button.click()

In [235]:
data = []

# 무한 루프: 다음 페이지가 없을 때까지 반복
while True:
    # 현재 페이지의 모든 포스트 수를 파악
    total_posts = len(driver.find_elements(By.CSS_SELECTOR, "#h_inner > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a"))

    # 각 포스트에 접근하여 정보 수집
    for i in range(1, total_posts + 1):
        # 동적 CSS 선택자 생성
        post_selector = f"#h_inner > div:nth-child({i}) > table > tbody > tr:nth-child(1) > td:nth-child(2) > a"
        
        # 해당 포스트를 선택
        post = driver.find_element(By.CSS_SELECTOR, post_selector)
        
        # 포스트 URL 저장
        post_url = post.get_attribute('href')
        
        # 포스트 클릭
        post.click()
        
        # 잠시 대기
        time.sleep(3)
        
        # 제목, 본문, 댓글, 날짜 등 필요한 정보 크롤링
        title = driver.find_element(By.CLASS_NAME, 'color_dark').text
        # 본문(content) 크롤링 (두 가지 클래스 중 하나를 찾음)

        content = driver.find_element(By.CLASS_NAME, 'content_body_padding').text
        if content == '':
            content = driver.find_element(By.CLASS_NAME, 'body_editor').text
        # 댓글 크롤링
        comments_elements = driver.find_elements(By.CLASS_NAME, 'comment_text')
        comments = [comment.text for comment in comments_elements]
        date = driver.find_element(By.CLASS_NAME, 'etc').text

        # 데이터 저장
        data.append({
            "title": title,
            "content": content,
            "comments": comments,
            "url": post_url,
            "date": date
        })

        # 이전 페이지로 돌아가기
        driver.back()
        time.sleep(3)

    # 다음 페이지로 이동
    try:
        next_button = driver.find_element(By.LINK_TEXT, '▶')
        next_button.click()
        time.sleep(3)
    except:
        print("마지막 페이지입니다.")
        break

# 드라이버 종료
driver.quit()

마지막 페이지입니다.


In [218]:
data

[{'title': '아니 시발 실화냐',
  'content': '학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강 총회랑 동아리 단체 모임이 다음주, 다다음주 목요일임\n\n2주 연속으로 행사 하나 하는거 때문에 2시간 전철 타고 가서 저녁에 술 먹고 다시 전철 2시간 타고 집에 와야됨 에바야',
  'comments': ['이야 이건 억까지', 'ㅋㅋ', '2시간거리를 통학해?!'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11201197',
  'date': '2024-08-31 22:42:27'},
 {'title': '조깅하는 사람있어?',
  'content': '이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까지 자고\n\n강의요일인 수목금은 아침 7시에 인나서 조깅할라하거든?\n\n근데 이렇게 불규칙하게 하면 생체리듬깨져서 건강 나빠짐?',
  'comments': ['몰라서 추..', '조깅해서 생체리듬 깨진다는 말은 못들어봤는데'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11188930',
  'date': '2024-08-24 18:47:37'},
 {'title': '학교 5과목 듣는데 무공강이라면 믿으시겠읍니까',
  'content': '',
  'comments': ['그거 완전 고등학생', '매일 걸을 수 있으니까 럭키비키잖앙'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11185642',
  'date': '2024-08-22 19:16:44'},
 {'title': '대학 강좌에 마블 시네마틱 유니버스 교양 강좌 있었는데',
  'content': '시간표 짜다가 아무래도 강의 하나 듣자고 공강 버리

In [352]:
# Pandas DataFrame으로 변환
df = pd.DataFrame(data)

# 여러 댓글을 한 셀에 합쳐서 저장 (줄바꿈 문자로 합치기)
df['comments'] = df['comments'].apply(lambda x: '\n'.join(x))

# CSV 파일로 저장 (utf-8-sig 인코딩 사용)
df.to_csv('fun_univ_original.csv', index=False, encoding='utf-8-sig')

In [354]:
# 이미 수집된 데이터를 하나의 제목-내용-댓글 형태로 변환
flattened_data = []

for entry in data:
    title = entry['title']
    content = entry['content']
    url = entry['url']
    date = entry['date']
    
    if entry['comments']:  # 댓글이 있는 경우
        for comment in entry['comments']:
            flattened_data.append({
                "title": title,
                "content": content,
                "comment": comment,
                "url": url,
                "date": date
            })
    else:  # 댓글이 없는 경우
        flattened_data.append({
            "title": title,
            "content": content,
            "comment": "",  # 빈 댓글로 저장
            "url": url,
            "date": date
        })

# Pandas DataFrame으로 변환
df = pd.DataFrame(flattened_data)

# CSV 파일로 저장 (utf-8-sig 인코딩 사용)
df.to_csv('fun_univ_flattened.csv', index=False, encoding='utf-8-sig')

In [356]:
# 날짜 2016-09-03 까지만 뽑기
df = pd.read_csv('fun_univ_flattened.csv')
df

,title,content,comment,url,date
0,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,이야 이건 억까지,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
1,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,ㅋㅋ,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
2,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,2시간거리를 통학해?!,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
3,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,몰라서 추..,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37
4,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,조깅해서 생체리듬 깨진다는 말은 못들어봤는데,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37
...,...,...,...,...,...
1831,진정한 대학생이 되기위한 대학생활 교범,"진정한 대학생이 되기 위한,\n\n대학생활 교범\n\n-저자 흑표범EX-\n\n\n...",웃유는 폼이냐??웃유는 폼이냐??웃유는 폼이냐??웃유는 폼이냐??웃유는 폼이냐??웃...,https://m.humoruniv.com/board/read.html?table=...,2004-10-11 16:58:25
1832,진정한 대학생이 되기위한 대학생활 교범,"진정한 대학생이 되기 위한,\n\n대학생활 교범\n\n-저자 흑표범EX-\n\n\n...",별로 공감은 안간다...웃기다기 보단 짜증난다...대학이 장난인가...,https://m.humoruniv.com/board/read.html?table=...,2004-10-11 16:58:25
1833,진정한 대학생이 되기위한 대학생활 교범,"진정한 대학생이 되기 위한,\n\n대학생활 교범\n\n-저자 흑표범EX-\n\n\n...",젠장.. 원한건 아닌데 ㅈ렇게 되어가구있잖아 -0-,https://m.humoruniv.com/board/read.html?table=...,2004-10-11 16:58:25
1834,진정한 대학생이 되기위한 대학생활 교범,"진정한 대학생이 되기 위한,\n\n대학생활 교범\n\n-저자 흑표범EX-\n\n\n...",'') 대학이 장난이라니 -_-;; 술먹고 놀자 대학생인게 더 장난인거아닌가 솔직히...,https://m.humoruniv.com/board/read.html?table=...,2004-10-11 16:58:25


In [358]:
# 먼저 '작성'이라는 텍스트를 제거
df['date'] = df['date'].str.replace('작성 ', '')

In [360]:
# 날짜를 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'])

In [362]:
# 날짜 필터링
start_date = datetime.strptime("2016-01-01", "%Y-%m-%d")
filtered_df = df[(df['date'] >= start_date) & (df['date'] <= datetime.today())]

In [422]:
# 필터링된 데이터 저장
filtered_df.to_csv('fun_univ_2016.csv', index=False, encoding='utf-8-sig')

filtered_df.tail()  # 필터링된 데이터의 첫 몇 줄 확인

,title,content,comment,url,date
1325,"아, 직장게 분들아",알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 1000원도 안하는 것들...,데미안 같은 건 사놓고 읽지도 않을 게 뻔함..,https://m.humoruniv.com/board/read.html?table=...,2016-03-15 20:37
1326,"아, 직장게 분들아",알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 1000원도 안하는 것들...,알바비 얼마 받아여?,https://m.humoruniv.com/board/read.html?table=...,2016-03-15 20:37
1327,금요일 공강 생각했던것보다,어마어마한 꿀인것같다\n무려 주말이 3일!,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-12 16:11
1328,빨리 노트북사고십다,공강시간에 노가다 해서 용도기 스펙업해줘야...,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-11 2:12
1329,D+4,오늘 학습 한 내용 : 한국사 영어 형법 [그동안 배운것 복습]\n\n지겨웠다 아직...,영어문제 1. love가 뭔지쓰시오,https://m.humoruniv.com/board/read.html?table=...,2016-03-10 22:16


In [424]:
filtered_df.tail(3)

,title,content,comment,url,date
1327,금요일 공강 생각했던것보다,어마어마한 꿀인것같다\n무려 주말이 3일!,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-12 16:11
1328,빨리 노트북사고십다,공강시간에 노가다 해서 용도기 스펙업해줘야...,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-11 2:12
1329,D+4,오늘 학습 한 내용 : 한국사 영어 형법 [그동안 배운것 복습]\n\n지겨웠다 아직...,영어문제 1. love가 뭔지쓰시오,https://m.humoruniv.com/board/read.html?table=...,2016-03-10 22:16


In [426]:
# CSV 파일 로드
filtered_df = pd.read_csv('fun_univ_2016.csv', encoding='utf-8-sig')

# 피클 파일로 저장
filtered_df.to_pickle('fun_univ_2016.pkl')

In [428]:
pickle = pd.read_pickle('fun_univ_2016.pkl')
pickle.head()

,title,content,comment,url,date
0,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,이야 이건 억까지,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42
1,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,ㅋㅋ,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42
2,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,2시간거리를 통학해?!,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42
3,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,몰라서 추..,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47
4,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,조깅해서 생체리듬 깨진다는 말은 못들어봤는데,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47


---

#### 전처리
- title, content 합치기
- unique값 comment 밑에 붙히기

In [370]:
new_df = filtered_df.copy()
new_df.head()

,title,content,comment,url,date
0,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,이야 이건 억까지,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
1,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,ㅋㅋ,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
2,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,2시간거리를 통학해?!,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27
3,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,몰라서 추..,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37
4,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,조깅해서 생체리듬 깨진다는 말은 못들어봤는데,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37


In [408]:
# title과 content를 결합하여 새로운 열 'title_content_combined' 생성
new_df['title_content_combined'] = new_df['title'].fillna('') + " " + new_df['content'].fillna('')
new_df

,title,content,comment,url,date,title_content_combined
0,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,이야 이건 억까지,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27,아니 시발 실화냐 학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 ...
1,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,ㅋㅋ,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27,아니 시발 실화냐 학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 ...
2,아니 시발 실화냐,학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강...,2시간거리를 통학해?!,https://m.humoruniv.com/board/read.html?table=...,2024-08-31 22:42:27,아니 시발 실화냐 학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 ...
3,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,몰라서 추..,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37,조깅하는 사람있어? 이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강...
4,조깅하는 사람있어?,이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까...,조깅해서 생체리듬 깨진다는 말은 못들어봤는데,https://m.humoruniv.com/board/read.html?table=...,2024-08-24 18:47:37,조깅하는 사람있어? 이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강...
...,...,...,...,...,...,...
1325,"아, 직장게 분들아",알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 1000원도 안하는 것들...,데미안 같은 건 사놓고 읽지도 않을 게 뻔함..,https://m.humoruniv.com/board/read.html?table=...,2016-03-15 20:37:03,"아, 직장게 분들아 알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 10..."
1326,"아, 직장게 분들아",알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 1000원도 안하는 것들...,알바비 얼마 받아여?,https://m.humoruniv.com/board/read.html?table=...,2016-03-15 20:37:03,"아, 직장게 분들아 알라딘에서 10년대여 Ebook 50% 추가할인해요. 권당 10..."
1327,금요일 공강 생각했던것보다,어마어마한 꿀인것같다\n무려 주말이 3일!,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-12 16:11:49,금요일 공강 생각했던것보다 어마어마한 꿀인것같다\n무려 주말이 3일!
1328,빨리 노트북사고십다,공강시간에 노가다 해서 용도기 스펙업해줘야...,NaN,https://m.humoruniv.com/board/read.html?table=...,2016-03-11 02:12:02,빨리 노트북사고십다 공강시간에 노가다 해서 용도기 스펙업해줘야...


In [430]:
new_df.to_csv('fun_univ_2016_combined.csv', index=False, encoding='utf-8-sig')

In [412]:
unique_combined = new_df['title_content_combined'].unique()

---

#### 테스트

In [228]:
data = []

# 현재 페이지의 모든 포스트 수를 파악
total_posts = len(driver.find_elements(By.CSS_SELECTOR, "#h_inner > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a"))

# 각 포스트에 접근하여 정보 수집
for i in range(1, total_posts + 1):
    # 동적 CSS 선택자 생성
    post_selector = f"#h_inner > div:nth-child({i}) > table > tbody > tr:nth-child(1) > td:nth-child(2) > a"
    
    # 해당 포스트를 선택
    post = driver.find_element(By.CSS_SELECTOR, post_selector)
    
    # 포스트 URL 저장
    post_url = post.get_attribute('href')
    
    # 포스트 클릭
    post.click()
    
    # 잠시 대기
    time.sleep(3)
    
    # 제목, 본문, 댓글, 날짜 등 필요한 정보 크롤링
    title = driver.find_element(By.CLASS_NAME, 'color_dark').text
    
    # 본문(content) 크롤링 (두 가지 클래스 중 하나를 찾음)
    content = driver.find_element(By.CLASS_NAME, 'content_body_padding').text
    if content == '':
        content = driver.find_element(By.CLASS_NAME, 'body_editor').text
    
    # 댓글 크롤링
    comments_elements = driver.find_elements(By.CLASS_NAME, 'comment_text')
    comments = [comment.text for comment in comments_elements]
    
    # 날짜 크롤링
    date = driver.find_element(By.CLASS_NAME, 'etc').text

    # 데이터 저장
    data.append({
        "title": title,
        "content": content,
        "comments": comments,
        "url": post_url,
        "date": date
    })

    # 이전 페이지로 돌아가기
    driver.back()
    time.sleep(3)

# 드라이버 종료
driver.quit()

In [230]:
data

[{'title': '아니 시발 실화냐',
  'content': '학교 강의 월수금 9시부터 6시까지 해 놓는 대신 화목 공강으로 해놨는데\n\n개강 총회랑 동아리 단체 모임이 다음주, 다다음주 목요일임\n\n2주 연속으로 행사 하나 하는거 때문에 2시간 전철 타고 가서 저녁에 술 먹고 다시 전철 2시간 타고 집에 와야됨 에바야',
  'comments': ['이야 이건 억까지', 'ㅋㅋ', '2시간거리를 통학해?!'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11201197',
  'date': '2024-08-31 22:42:27'},
 {'title': '조깅하는 사람있어?',
  'content': '이제 개강이라 아침에 조깅하고 학교가려는데\n\n월화토일은 공강이라 9-10시? 까지 자고\n\n강의요일인 수목금은 아침 7시에 인나서 조깅할라하거든?\n\n근데 이렇게 불규칙하게 하면 생체리듬깨져서 건강 나빠짐?',
  'comments': ['몰라서 추..', '조깅해서 생체리듬 깨진다는 말은 못들어봤는데'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11188930',
  'date': '2024-08-24 18:47:37'},
 {'title': '학교 5과목 듣는데 무공강이라면 믿으시겠읍니까',
  'content': '무려 학교에서 공식으로 짜준 시간표라면\n믿을 수 있겠습니까\n\n이게 말이 되냐거~~~~',
  'comments': ['그거 완전 고등학생', '매일 걸을 수 있으니까 럭키비키잖앙'],
  'url': 'https://m.humoruniv.com/board/read.html?table=pdswait&number=11185642',
  'date': '2024-08-22 19:16:44'},
 {'title': '대학 강좌에 마블 시네마틱 유니버스 교양 강좌